In [1]:
import numpy as np
import pandas as pd
import anndata
import matplotlib.pyplot as plt
import spatialdata as sd
import scanpy as sc
import seaborn as sns

In [ ]:
adata = sc.read_h5ad('combinedall.h5ad')
adata

In [79]:
import numpy as np

rbRNA = adata.layers['rbRNA'] 
transcription = adata.X  

transcription_safe = np.where(transcription == 0, np.nan, transcription)

te = rbRNA / transcription_safe
adata.layers['TE'] = te

In [81]:
totalRNA_cell_sum = np.sum(adata.X, axis=1) 
rbRNA_cell_sum = np.sum(adata.layers['rbRNA'], axis=1)  
te_per_cell_mean = rbRNA_cell_sum / totalRNA_cell_sum       

adata.obs['TE_cell_mean'] = te_per_cell_mean

totalRNA_gene_sum = np.sum(adata.X, axis=0) 
rbRNA_gene_sum = np.sum(adata.layers['rbRNA'], axis=0)  
te_per_gene_mean = rbRNA_gene_sum / totalRNA_gene_sum         

adata.var['TE_gene_mean'] = te_per_gene_mean


### QC

In [82]:
import scanpy as sc
import pandas as pd

sc.pp.calculate_qc_metrics(adata, percent_top=None,inplace=True)

qc_totalRNA_obs = adata.obs.copy()
qc_totalRNA_var = adata.var.copy()
qc_totalRNA_obs.columns = [f'totalRNA_{col}' for col in qc_totalRNA_obs.columns]
qc_totalRNA_var.columns = [f'totalRNA_{col}' for col in qc_totalRNA_var.columns]

temp_adata = adata.copy() 
temp_adata.X = adata.layers['rbRNA']  
sc.pp.calculate_qc_metrics(temp_adata, percent_top=None,inplace=True)

qc_rbRNA_obs = temp_adata.obs.copy()
qc_rbRNA_var = temp_adata.var.copy()
qc_rbRNA_obs.columns = [f'rbRNA_{col}' for col in qc_rbRNA_obs.columns]
qc_rbRNA_var.columns = [f'rbRNA_{col}' for col in qc_rbRNA_var.columns]


adata.obs = pd.concat([qc_totalRNA_obs, qc_rbRNA_obs], axis=1)

adata.var = pd.concat([qc_totalRNA_var, qc_rbRNA_var], axis=1)



### filter

In [83]:
# Calculate max count for each gene
adata.var['max_counts'] = adata.X.max(axis=0)

In [85]:
# Remove rbRNA columns
adata.obs = adata.obs.drop(columns=[
    'rbRNA_cell_id', 
    'rbRNA_area',
    'rbRNA_region', 
    'rbRNA_condition',
    'rbRNA_sample'
])

# Rename totalRNA columns
adata.obs = adata.obs.rename(columns={
    'totalRNA_cell_id': 'cell_id',
    'totalRNA_area': 'area', 
    'totalRNA_region': 'region',
    'totalRNA_condition': 'condition',
    'totalRNA_sample': 'sample'
})


In [86]:
adata.obs['totalRNA_density'] = adata.obs['totalRNA_total_counts'] / adata.obs['area']
adata.obs['rbRNA_density'] = adata.obs['rbRNA_total_counts'] / adata.obs['area']

In [ ]:
# Plot top 20 most expressed genes 
sc.pl.highest_expr_genes(adata, n_top=20)

In [ ]:
# filter by volume
sns.histplot(data=adata.obs, x='area')
thres_vol_lower = 0.5e6
thres_vol_higher = 6e6
plt.axvline(x=thres_vol_lower, c='slategrey')
plt.axvline(x=thres_vol_higher, c='slategrey')

In [89]:
pass_vol = [1 if area<thres_vol_higher and area>thres_vol_lower else 0 for area in adata.obs['area']]
adata.obs['pass_volume_filter'] = pass_vol

In [ ]:
## filter cells
import matplotlib.pyplot as plt
import seaborn as sns

sns.histplot(data=adata.obs, x='totalRNA_total_counts', linewidth=0, label='totalRNA', color='#1f77b4')
thres_tr_lower_totalRNA = 500
thres_tr_higher_totalRNA = 6000
plt.axvline(x=thres_tr_lower_totalRNA, c='#1f77b4')
plt.axvline(x=thres_tr_higher_totalRNA, c='#1f77b4')

sns.histplot(data=adata.obs, x='rbRNA_total_counts', linewidth=0, label='rbRNA', color='#ff7f0e')


plt.xlabel('Total Reads Counts')

plt.legend(loc='upper right')

plt.show()

In [92]:

pass_tr = [
    1 if totalRNA_total_counts > thres_tr_lower_totalRNA and totalRNA_total_counts < thres_tr_higher_totalRNA
    else 0
    for totalRNA_total_counts in adata.obs['totalRNA_total_counts']
]
adata.obs['pass_counts_filter'] = pass_tr

In [93]:
adata.obs['pass_two_filters'] = np.logical_and(adata.obs['pass_volume_filter'], adata.obs['pass_counts_filter'])

In [ ]:
import pandas as pd
filtered_data = adata.obs[adata.obs['pass_two_filters']]
totalRNA_data = filtered_data[['totalRNA_density']].rename(columns={'totalRNA_density': 'density'})
totalRNA_data['source'] = 'totalRNA'

rbRNA_data = filtered_data[['rbRNA_density']].rename(columns={'rbRNA_density': 'density'})
rbRNA_data['source'] = 'rbRNA'

combined_data = pd.concat([totalRNA_data, rbRNA_data])

sns.histplot(data=combined_data, x='source', y='density', hue='source', multiple='dodge', kde=False)


plt.legend()
plt.xlabel('Source')
plt.ylabel('Density')
plt.title('Density Comparison: totalRNA vs rbRNA')

In [ ]:
## filter density
dens_thres = 0
pass_dens = [
    1 if (rbRNA_density > dens_thres) and
         (totalRNA_density > dens_thres) 
    else 0
    for rbRNA_density, totalRNA_density in zip(adata.obs['rbRNA_density'], adata.obs['totalRNA_density'])
]
adata.obs['pass_density_filter'] = pass_dens


# check density after filtering
sns.histplot(data=adata.obs[adata.obs['pass_two_filters']],  y='totalRNA_density', linewidth=0)


# check density after filtering
sns.histplot(data=adata.obs[adata.obs['pass_two_filters']],  y='rbRNA_density', linewidth=0)

plt.ylabel('Density')
plt.title('Density Comparison: totalRNA vs rbRNA')


In [96]:
adata.obs['pass_all_filters'] = np.logical_and(adata.obs['pass_two_filters'], adata.obs['pass_density_filter'])

In [ ]:
import pandas as pd
filtered_data = adata.obs[adata.obs['pass_all_filters']]
totalRNA_data = filtered_data[['totalRNA_total_counts']].rename(columns={'totalRNA_total_counts': 'total_counts'})
totalRNA_data['source'] = 'totalRNA'

rbRNA_data = filtered_data[['rbRNA_total_counts']].rename(columns={'rbRNA_total_counts': 'total_counts'})
rbRNA_data['source'] = 'rbRNA'

combined_data = pd.concat([totalRNA_data, rbRNA_data])

sns.violinplot(data=combined_data, x='source', y='total_counts', hue='source')


plt.legend()
plt.xlabel('Source')
plt.ylabel('total_counts')
plt.title('Pass all cell filters: total_counts')

In [ ]:
total_cells = len(adata.obs)
filtered_cells = np.sum(adata.obs['pass_all_filters'])
filtered_out = total_cells - filtered_cells

print(f"Total cells before filtering: {total_cells}")
print(f"Cells passing all filters: {filtered_cells}")
print(f"Filtered out cells: {filtered_out} ({filtered_out/total_cells*100:.2f}%)")

In [ ]:
# low abundance genes
plt.hist([np.mean(adata.X[adata.obs['pass_all_filters']], axis=0), 
        np.mean(adata.layers['rbRNA'][adata.obs['pass_all_filters']], axis=0)], 
        range=(0,8), bins=50, log=True, histtype='step', rwidth=1,
        label=['totalRNA', 'rbRNA'] )
plt.xlabel('mean expression (lower end, after cell filtering)'), plt.legend()

In [ ]:
## filter genes


def filter_genes(data, pct_threshold, max_threshold, var_key, data_name):
    f1 = np.count_nonzero(data, axis=0) > pct_threshold * data.shape[0]  
    f2 = np.amax(data, axis=0) > max_threshold  
    f = np.logical_and(f1, f2)  
    print(
        f"{data_name} threshold: expressed in at least {pct_threshold * 100:.1f}% cells AND max expression in a cell greater than {max_threshold}"
    )
    print(
        f"Filtered out: {adata.n_vars - np.count_nonzero(f)} genes -- {(adata.n_vars - np.count_nonzero(f)) / adata.n_vars * 100:.2f}%"
    )
    adata.var[var_key] = f

    filtered_genes = adata.var_names[~f]
    print(f"Filtered genes for {data_name}: {filtered_genes}")

totalRNA = adata.X[adata.obs['pass_all_filters']] 
filter_genes(totalRNA, 0.05, 4, 'filter_totalRNA', 'totalRNA')


In [101]:
adata.var.rename(columns={'filter_totalRNA': 'filter'}, inplace=True)

In [ ]:
# Count number of cells passing all filters
n_passing = np.sum(adata.obs['pass_all_filters'])
n_total = len(adata.obs)
pct_passing = n_passing / n_total * 100

print(f"Number of cells passing all filters: {n_passing} out of {n_total} ({pct_passing:.1f}%)")

# Display full dataframe
adata.obs

In [ ]:
# Count number of genes passing filters
n_passing = np.sum(adata.var['filter'])
n_total = len(adata.var)
pct_passing = n_passing / n_total * 100

print(f"Number of genes passing all filters: {n_passing} out of {n_total} ({pct_passing:.1f}%)")

# Display full dataframe
adata.var

In [104]:
# Remove rbRNA columns
adata.obs = adata.obs.drop(columns=[
    'rbRNA_TE_cell_mean'
])

# Rename totalRNA columns
adata.obs = adata.obs.rename(columns={
    'totalRNA_TE_cell_mean': 'TE_cell_mean'
})


In [105]:

# Remove rbRNA columns
adata.var = adata.var.drop(columns=[
    'rbRNA_TE_gene_mean'
])

# Rename totalRNA columns
adata.var = adata.var.rename(columns={
    'totalRNA_TE_gene_mean': 'TE_gene_mean'
})


In [106]:
adata.write_h5ad('/storage/lingyuan2/20250101rj/downstream/alldatafinalzarr/filtered.h5ad')

In [ ]:
obs_filter = adata.obs['pass_all_filters']
var_filter = adata.var['filter']

filtered_data = adata[obs_filter, var_filter]

print(filtered_data)

In [108]:
filtered_data.write_h5ad('/storage/lingyuan2/20250101rj/downstream/alldatafinalzarr/filtered_data.h5ad')